<a href="https://colab.research.google.com/github/Suprim-Shakya/BERTify/blob/main/Summify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymupdf
import pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 28.1 MB/s eta 0:00:00


In [2]:
print(pymupdf.__doc__)

PyMuPDF 1.25.2: Python bindings for the MuPDF 1.25.2 library (rebased implementation).
Python 3.11 running on linux (64-bit).



In [4]:
def extract_paper_text(pdf_path):
  text = ""
  doc = pymupdf.open(pdf_path)
  for page in doc:
    text += page.get_text()
  return text

pdf_path = "paper.pdf"
cleaned_text = extract_paper_text(pdf_path)


In [5]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

# Load the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Check if a GPU is available and move the model to GPU if it is
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def summarize_text(text, model, tokenizer, max_chunk_size=1024):
    chunks = [text[i:i+max_chunk_size] for i in range(0, len(text), max_chunk_size)]
    summaries = []
    for chunk in chunks:
        inputs = tokenizer(chunk, max_length=max_chunk_size, return_tensors="pt", truncation=True)

        # Move input tensors to the same device as the model
        inputs = {key: value.to(device) for key, value in inputs.items()}

        summary_ids = model.generate(inputs["input_ids"], max_length=200, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
        summaries.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True))
    return " ".join(summaries)

# Example usage
summary = summarize_text(cleaned_text, model, tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [8]:
def hierarchical_summarization(text, model, tokenizer, max_chunk_size=1024):
  first_summary = summarize_text(text, model, tokenizer, max_chunk_size)

  inputs = tokenizer(first_summary, max_length=max_chunk_size, return_tensors="pt", truncation=True)

  inputs = {key: value.to(device) for key, value in inputs.items()}

  summary_ids = model.generate(inputs["input_ids"], max_length=200, min_length=50, length_penalty=2.0, num_beams = 4, early_stopping=True)

  final_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return final_summary

hierarchical_summary = hierarchical_summarization(cleaned_text, model, tokenizer)

In [9]:
print(hierarchical_summary)

Large Language Models (LLMs) have recently demonstrated remarkable capabilities in natural language processing tasks and beyond. The need for generalized models stems from the growing demand for machines to handle complex language tasks. This article provides an overview of the literature on a broadrange of LLM-related concepts. It also covers the advanced topics at the frontier of research in LLMs.
